In [ ]:
!pip3 install --upgrade pip
!pip3 install --upgrade pandas
!pip3 install --upgrade sklearn
!pip3 install --upgrade pymysql
!pip3 install --upgrade pytz

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None 
from keras.models import load_model
import requests
import json
import datetime
import pymysql
import pytz

In [ ]:
# database connection init
conn=pymysql.connect(
    host='sql10.freemysqlhosting.net',
    port=int(3306),
    user='sql10489794',
    passwd='IPELfw5A2X',
    db='sql10489794'
)

In [ ]:
# fetch weather data from visualcrossing.com
time_now = datetime.datetime.now(tz=pytz.timezone("Europe/Berlin"))
key = "JPRR7UQ66H89YQ23GM3TYFWBU"
date_start = (time_now - datetime.timedelta(days=2)).strftime("%Y-%m-%d") 
date_stop  = (time_now).strftime("%Y-%m-%d")
result = requests.get(f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/krak%C3%B3w/{date_start}/{date_stop}?unitGroup=metric&include=hours&key={key}&contentType=json")
json_dict = json.loads(str(result.content.decode('utf-8')))
new_dict = {}

# reshape dictionary for further processing
i = 0
for day in json_dict['days'][::-1]:
    for hour in day['hours'][::-1]:
        # leave only relavant data
        if day['datetime'] != time_now.strftime("%Y-%m-%d") or hour['datetime'] < time_now.strftime("%H:%M:00"):
            new_dict[i] = hour.copy()
            # set time in specific format
            new_dict[i]['datetime'] = day['datetime'] + 'T' + hour['datetime']
            # fetch weather measurements from arduino database
            query = f"SELECT * FROM esp WHERE weather_time = '{day['datetime']}' AND hour = '{hour['datetime'].split(':')[0]}'"
            esp_measurement=pd.read_sql_query(query, conn)
            if not esp_measurement.empty:
                print(esp_measurement)
                new_dict[i]['temp'] = esp_measurement['temperature'][0]
                new_dict[i]['humidity'] = esp_measurement['humidity'][0]
            i=i+1

# change shape            
measured_data_wlabels = pd.DataFrame.from_dict(new_dict).transpose()[:48][::-1]

In [ ]:
measured_data_wlabels

In [ ]:
columns_names = [col for col in measured_data_wlabels]

pd.DataFrame(columns_names, columns=['column_name'])

In [ ]:
import joblib
sc = joblib.load("scaler.save")

In [ ]:
selected_columns = [2, 4, 21]
selected_columns_names = [columns_names[col] for col in selected_columns]
print(selected_columns_names)

measured_data_wlabels = measured_data_wlabels.iloc[:,selected_columns]

# perform one-hot encoding of weather conditions
cond = ['Snow', 'Rain, Partially cloudy', 'Snow, Partially cloudy', 'Rain', 'Partially cloudy', 'Rain, Overcast', 'Snow, Rain, Overcast', 'Overcast', 'Snow, Overcast', 'Clear', 'Snow, Rain, Partially cloudy']
for label in cond:
    measured_data_wlabels[label] = 0
    measured_data_wlabels.loc[measured_data_wlabels['conditions']==label, label] = 1
measured_data_wlabels = measured_data_wlabels.drop(columns='conditions')

measured_data = measured_data_wlabels.values

measured_data = sc.transform(measured_data)
measured_data = np.array(measured_data)
measured_data = np.reshape(measured_data,(1,measured_data.shape[1],measured_data.shape[0]))

In [ ]:
!tar xzf weather_forecast_model.tgz

In [ ]:
model = load_model('weather_forecast_model')

In [ ]:
# predict weather with precomputed model
predicted_weather = model.predict(measured_data)
predicted_weather = np.reshape(predicted_weather, (predicted_weather.shape[2],predicted_weather.shape[1]))
predicted_weather = sc.inverse_transform(predicted_weather)

In [ ]:
# restore conditions from one-hot encoding
predicted_weather_wlabels = pd.DataFrame(predicted_weather, columns=[col for col in selected_columns_names if col != 'conditions']+cond)
max_cond = np.argmax(predicted_weather[:, 2:], axis=1)
for i, mcond in enumerate(max_cond):
    predicted_weather_wlabels.loc[i, 'conditions'] = cond[mcond]
predicted_weather_wlabels = predicted_weather_wlabels.drop(columns=cond)


In [ ]:
# add datetime for storing purposes
predicted_weather_wlabels['datetime'] = [(time_now + datetime.timedelta(hours=i+1)).strftime("%Y-%m-%dT%H:00:00") for i in range(12)]
column_to_move = predicted_weather_wlabels.pop("datetime")
predicted_weather_wlabels.insert(0, "datetime", column_to_move)
predicted_weather_wlabels

In [ ]:
# push forecast to remote database
cursor = conn.cursor()
query = "INSERT INTO forecast (date, hour, temp, humidity, conditions) VALUES " + ", ".join(["('{}', '{}', {}, {}, '{}')".format(row[1]['datetime'].split("T")[0], row[1]['datetime'].split("T")[1], int(row[1]['temp']), int(row[1]['humidity']), row[1]['conditions']) for row in predicted_weather_wlabels[::-1].iterrows()])
print(query)
cursor.execute(query)
conn.commit()
cursor.execute(f"DELETE FROM forecast WHERE id < {cursor.lastrowid}")
conn.commit()